In [1]:
# Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random
import os
import IPython.display as ipd
from sklearn.model_selection import train_test_split
from tqdm import trange
from tqdm.notebook import tqdm
from pathlib import Path
from datetime import datetime
import logging

# Torch imports
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim

# SBERT imports 
from datasets import Dataset, load_dataset
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments, CrossEncoder, LoggingHandler
from sentence_transformers.cross_encoder.evaluation import CEBinaryAccuracyEvaluator, CEBinaryClassificationEvaluator
from sentence_transformers.training_args import BatchSamplers
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer


In [2]:
DEVICE = None
if torch.cuda.is_available():
    DEVICE = "cuda"
elif torch.backends.mps.is_available():
    DEVICE = "mps"
else:
    DEVICE = "cpu"
print(f"Using {DEVICE}")

Using cuda


In [3]:
# Convenience and saving flags
ABRIDGED_RUN = False
FREEZE_EMBEDDING = True

# Training hyperparameters
BATCH_SIZE = 16 # Number of samples per batch while training our network
NUM_EPOCHS = 10 # Number of epochs to train our network
WARMUP_PROPORTION = 0.5 # Number of epochs to use as warmup steps

if ABRIDGED_RUN == True:
    BATCH_SIZE = 16
    NUM_EPOCHS = 50

LEARNING_RATE = 2e-5

TRAINING_DATA_SOURCES = ['wiki', 'grover'] # ['reviews', 'wiki', 'grover']

MODEL_NAME = "SBERT_CrossEncoder_all-mpnet-base-v2_"

# Directories
OUTPUT_DIR = Path(f"training_results/{"baseline_" if FREEZE_EMBEDDING else ""}{"_".join(TRAINING_DATA_SOURCES)}_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}") # Checkpoints, models, and statistics will be saved here
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = Path("../raw data/")

In [4]:
data = pd.read_csv("../raw data/combined_data.csv")
data = data[data['Original dataset'] != 'essays'] # Throw out essays data
data['Stratify'] = data['Label'] + " " + data['Original dataset']

if ABRIDGED_RUN == True:
    data = data.sample(320)

# Set integer labels
# Human = 0, 1 = Machine
data['Label'] = data['Label'].apply(lambda x: 0 if x == 'Human' else 1)

# Train test split, stratified by binary labels
data_train, data_test = train_test_split(data, test_size = 0.2, stratify=data['Stratify'], random_state = 406)
train_tt, train_vv = train_test_split(data_train, test_size = 0.2, stratify=data_train['Stratify'], random_state = 406)
train_tt = train_tt[train_tt['Original dataset'].isin(TRAINING_DATA_SOURCES)]

In [5]:
train_dataset = Dataset.from_dict({
    "text": train_tt['Text'],
    "label": train_tt['Label'],
})

In [6]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)

In [7]:
# Copied from https://github.com/UKPLab/sentence-transformers/blob/7290448809cb73f08f63c955550815775434beb4/sentence_transformers/cross_encoder/CrossEncoder.py#L143
def smart_batching_collate(batch):
        texts = []
        labels = []

        for example in batch:
            texts.append(example['text'])
            labels.append(example['label'])

        tokenized = tokenizer(
            texts, padding=True, truncation="longest_first", return_tensors="pt"
        )
        labels = torch.tensor(labels, dtype=torch.long).to(DEVICE)

        for name in tokenized:
            tokenized[name] = tokenized[name].to(DEVICE)

        return tokenized, labels

In [8]:
train_dataloader.collate_fn = smart_batching_collate

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-mpnet-base-v2",
                                            num_labels = 1).to(DEVICE)
if FREEZE_EMBEDDING:
    for p in model.mpnet.parameters():
        p.requires_grad = False

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

/home/amzi/installs/miniforge3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.classifier.parameters() if FREEZE_EMBEDDING else model.parameters(), lr = LEARNING_RATE)
scheduler = SentenceTransformer._get_scheduler(optimizer, scheduler = 'WarmupLinear', warmup_steps = WARMUP_PROPORTION * len(train_dataloader), t_total = len(train_dataloader)*NUM_EPOCHS)

In [12]:
def validation_accuracy(model, tokenizer, validation_samples):
    strata = ['reviews', 'wiki', 'grover']
    accuracies = {strat: 0 for strat in strata}
    for strat in strata:
        strat_validation_samples = validation_samples[validation_samples['Original dataset'] == strat]
        for index, sample in strat_validation_samples.iterrows():
            features = tokenizer(sample['Text'], padding=True, truncation="longest_first", return_tensors="pt")
            for key in features:
                features[key] = features[key].to(DEVICE)
            logits = model(**features, return_dict = True).logits
            prob = torch.nn.Sigmoid()(logits).item()
            label = sample['Label']
            accuracies[strat] += int(round(prob) == label)
    for strat in strata:
        accuracies[strat] = accuracies[strat]/(validation_samples['Original dataset'] == strat).sum()
    return accuracies

In [13]:
with open(OUTPUT_DIR/'accuracies.csv', 'w') as f:
    f.write('epoch, grover, wiki, reviews\n')
    with torch.no_grad():
        acc = validation_accuracy(model, tokenizer, train_vv)
        f.write(f'-1, {acc['grover']}, {acc['wiki']}, {acc['reviews']}\n')

for epoch in tqdm(range(NUM_EPOCHS), position = 0):
    for features, labels in tqdm(train_dataloader, position = 1, leave = False):
        logits = model(**features, return_dict=True).logits
        loss = criterion(logits, labels.unsqueeze(1).float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    with torch.no_grad():
        acc = validation_accuracy(model, tokenizer, train_vv)
    with open(OUTPUT_DIR/'accuracies.csv', 'a') as f:
        f.write(f'{epoch}, {acc['grover']}, {acc['wiki']}, {acc['reviews']}\n')
        
    scheduler.step()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

...

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

In [14]:
with torch.no_grad():
    acc = validation_accuracy(model, tokenizer, data_test)
with open(OUTPUT_DIR/'accuracies.csv', 'a') as f:
    f.write(f'TEST SET, {acc['grover']}, {acc['wiki']}, {acc['reviews']}\n')
print(f"""Accuracies on held out test set:
    grover: {100*acc['grover']:.2f}
    wiki: {100*acc['wiki']:.2f}
    reviews: {100*acc['reviews']:.2f}""")

Accuracies on held out test set:
    grover: 51.10
    wiki: 54.87
    reviews: 52.95
